# Sistema de Análisis Visual Dinámico para Imputación por Correlación

## Descripción del Sistema

Este notebook implementa un sistema avanzado de análisis visual para evaluar y comparar modelos de imputación por correlación en datasets de aeronaves. El sistema incluye:

### 🎯 **Funcionalidades Principales:**
- **Doble Gráfica Dinámica**: Comparación entre tipos de modelos vs análisis intra-tipo
- **Panel de Métricas Interactivo**: MAPE, R², Confianza con validación cruzada
- **Análisis de Residuos**: Detección de outliers y distribución de errores
- **Selector de Criterios**: Diferentes métodos de ranking de modelos
- **Recomendaciones Automáticas**: Alertas y sugerencias de mejora

### 📊 **Estructura de Análisis:**
1. **Gráfica Izquierda**: Comparación entre tipos (Linear, Poly, Log, Pot, Exp)
2. **Gráfica Derecha**: Análisis detallado dentro del tipo seleccionado
3. **Panel Inferior**: Métricas comparativas y análisis de decisión

### 🔧 **Tecnologías Utilizadas:**
- **Plotly**: Gráficas interactivas 2D/3D
- **IPyWidgets**: Controles dinámicos
- **Pandas**: Manipulación de datos
- **NumPy**: Cálculos numéricos
- **Scikit-learn**: Modelos y métricas

In [14]:
# ================================================================================
# SECCIÓN 1: IMPORTAR LIBRERÍAS NECESARIAS
# ================================================================================

# Librerías fundamentales
import pandas as pd
import numpy as np
import json
import pickle
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

# Librerías para visualización
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Librerías para interfaz interactiva
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Librerías para análisis estadístico
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.model_selection import LeaveOneOut
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# Importar módulos locales
import sys
sys.path.append('Modulos')
from imputacion_correlacion import (
    imputaciones_correlacion, 
    entrenar_modelo, 
    cargar_y_validar_datos,
    seleccionar_predictores_validos,
    generar_combinaciones,
    filtrar_mejores_modelos,
    validar_con_loocv
)

print("✅ Todas las librerías importadas exitosamente")
print("📦 Versiones principales:")
print(f"   - Pandas: {pd.__version__}")
print(f"   - NumPy: {np.__version__}")
import plotly
print(f"   - Plotly: {plotly.__version__}")
print(f"   - IPython: Widgets disponibles")

✅ Todas las librerías importadas exitosamente
📦 Versiones principales:
   - Pandas: 2.2.3
   - NumPy: 2.1.3
   - Plotly: 5.24.1
   - IPython: Widgets disponibles


In [ ]:
# ================================================================================
# SECCIÓN 2: DEFINIR FUNCIONES PRINCIPALES DEL SISTEMA
# ================================================================================

class AnalizadorModelos:
    """
    Clase principal para analizar modelos de imputación por correlación.
    
    MODOS DE FUNCIONAMIENTO:
    - Development: Carga desde Excel y reentrena modelos para testing rápido
    - Production: Carga desde diccionarios generados por imputation_loop.py
    """
    
    def __init__(self):
        self.df_original = None
        self.datos_tipo = {}
        self.diccionarios_modelos = {}
        self.df_resultado = None
        self.modo_carga = "sin_cargar"  # "desarrollo", "produccion", "sin_cargar"
        
    def cargar_desde_bucle_imputacion(self, df_original: pd.DataFrame, detalles_para_excel: List[Dict], diccionarios_modelos: Dict = None) -> pd.DataFrame:
        """
        🎯 FUNCIÓN PRINCIPAL PARA PRODUCCIÓN
        
        Carga datos desde los diccionarios generados por imputation_loop.py.
        Solo analiza imputaciones por correlación, excluyendo similitud y promedios.
        
        Args:
            df_original: DataFrame original antes de imputación
            detalles_para_excel: Lista de detalles de imputación (para filtrar correlación)
            diccionarios_modelos: Dict con modelos entrenados {clave_celda: {mejor_modelo, todos_los_modelos, ...}}
            
        Returns:
            DataFrame con información de carga o None si hay error
        """
        
        print("🚀 CARGANDO DESDE BUCLE DE IMPUTACIÓN (MODO PRODUCCIÓN)")
        print("=" * 55)
        
        try:
            # Validar inputs
            if df_original is None or df_original.empty:
                print("❌ DataFrame original no válido")
                return None
                
            if diccionarios_modelos is None:
                print("❌ Diccionarios de modelos no proporcionados")
                return None
                
            if not isinstance(diccionarios_modelos, dict):
                print("❌ Diccionarios de modelos deben ser un diccionario")
                return None
            
            self.df_original = df_original.copy()
            self.modo_carga = "produccion"
            
            print(f"📊 DataFrame original: {df_original.shape}")
            print(f"🔧 Diccionarios recibidos: {len(diccionarios_modelos)}")
            
            # Filtrar solo imputaciones por correlación
            if detalles_para_excel:
                celdas_correlacion = set()
                for detalle in detalles_para_excel:
                    if detalle.get("Método predictivo") == "Correlacion":
                        aeronave = detalle.get("Aeronave")
                        parametro = detalle.get("Parámetro")
                        if aeronave is not None and parametro is not None:
                            clave = f"aeronave_{aeronave}_parametro_{parametro}"
                            celdas_correlacion.add(clave)
                
                print(f"🎯 Celdas por correlación identificadas: {len(celdas_correlacion)}")
                
                # Filtrar diccionarios solo para correlación
                diccionarios_filtrados = {}
                for clave, datos in diccionarios_modelos.items():
                    if clave in celdas_correlacion:
                        diccionarios_filtrados[clave] = datos
                        
                self.diccionarios_modelos = diccionarios_filtrados
                print(f"✅ Diccionarios filtrados: {len(diccionarios_filtrados)} celdas por correlación")
                
            else:
                # Si no hay detalles, usar todos los diccionarios
                self.diccionarios_modelos = diccionarios_modelos.copy()
                print("⚠️ Sin detalles para filtrar - usando todos los diccionarios")
            
            if not self.diccionarios_modelos:
                print("❌ No hay diccionarios de correlación disponibles")
                return None
            
            # Procesar cada diccionario
            print("\n🔄 Procesando diccionarios...")
            diccionarios_validos = 0
            diccionarios_con_error = 0
            
            for clave, datos in self.diccionarios_modelos.items():
                try:
                    # Validar estructura del diccionario
                    if not isinstance(datos, dict):
                        print(f"  ❌ {clave}: Datos no válidos")
                        diccionarios_con_error += 1
                        continue
                    
                    mejor_modelo = datos.get("mejor_modelo")
                    todos_los_modelos = datos.get("todos_los_modelos", [])
                    
                    if mejor_modelo is None:
                        error_msg = datos.get("error", "Sin error especificado")
                        print(f"  ⚠️ {clave}: Sin mejor modelo ({error_msg})")
                        diccionarios_con_error += 1
                        continue
                    
                    # Validar que tenga al menos el modelo elegido
                    if not todos_los_modelos:
                        print(f"  ⚠️ {clave}: Sin lista de modelos - usando solo el mejor")
                        datos["todos_los_modelos"] = [mejor_modelo]
                    
                    diccionarios_validos += 1
                    print(f"  ✅ {clave}: {mejor_modelo.get('tipo', 'unknown')} - R²={mejor_modelo.get('r2', 0):.3f}")
                    
                except Exception as e:
                    print(f"  ❌ {clave}: Error procesando - {e}")
                    diccionarios_con_error += 1
            
            print(f"\n📊 RESUMEN:")
            print(f"  ✅ Diccionarios válidos: {diccionarios_validos}")
            print(f"  ❌ Diccionarios con error: {diccionarios_con_error}")
            print(f"  🎯 Total procesable: {diccionarios_validos}")
            
            if diccionarios_validos == 0:
                print("❌ No hay diccionarios válidos para analizar")
                return None
            
            # Generar datos_tipo para la interfaz
            self._generar_datos_tipo_desde_diccionarios()
            
            print("✅ CARGA DESDE BUCLE COMPLETADA")
            print(f"🎮 Listo para análisis visual con {diccionarios_validos} celdas")
            
            # Crear DataFrame resumen
            resumen_data = []
            for clave, datos in self.diccionarios_modelos.items():
                if datos.get("mejor_modelo"):
                    mejor = datos["mejor_modelo"]
                    resumen_data.append({
                        "Celda": clave,
                        "Mejor_Modelo": mejor.get("tipo", "unknown"),
                        "R2": mejor.get("r2", 0),
                        "MAPE": mejor.get("mape", 0),
                        "Confianza": mejor.get("Confianza", 0),
                        "Total_Modelos": len(datos.get("todos_los_modelos", []))
                    })
            
            return pd.DataFrame(resumen_data)
            
        except Exception as e:
            print(f"❌ Error cargando desde bucle: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def _generar_datos_tipo_desde_diccionarios(self):
        """
        Genera la estructura datos_tipo desde los diccionarios cargados.
        Adapta el formato para que sea compatible con el visualizador.
        """
        
        self.datos_tipo = {}
        
        for clave, datos in self.diccionarios_modelos.items():
            try:
                # Extraer aeronave y parámetro de la clave
                parts = clave.split("_")
                if len(parts) >= 4:
                    aeronave_idx = int(parts[1])
                    parametro = "_".join(parts[3:])  # En caso de que el parámetro tenga underscores
                else:
                    continue
                
                # Obtener modelos
                mejor_modelo = datos.get("mejor_modelo")
                todos_los_modelos = datos.get("todos_los_modelos", [])
                
                if mejor_modelo is None:
                    continue
                
                # Agrupar modelos por tipo
                modelos_por_tipo = {}
                for modelo in todos_los_modelos:
                    if modelo and isinstance(modelo, dict):
                        tipo = modelo.get("tipo", "unknown")
                        if tipo not in modelos_por_tipo:
                            modelos_por_tipo[tipo] = []
                        modelos_por_tipo[tipo].append(modelo)
                
                # Si no hay modelos agrupados, usar solo el mejor
                if not modelos_por_tipo:
                    tipo_mejor = mejor_modelo.get("tipo", "linear")
                    modelos_por_tipo[tipo_mejor] = [mejor_modelo]
                
                # Crear entrada en datos_tipo
                entrada_celda = {
                    "mejor_modelo": mejor_modelo,
                    "todos_los_modelos": todos_los_modelos,
                    "modelos_por_tipo": modelos_por_tipo,
                    "df_filtrado": datos.get("df_filtrado"),
                    "predictores": datos.get("predictores", []),
                    "familia_usada": datos.get("familia_usada", "sin_filtro"),
                    "modo_origen": "bucle_principal"
                }
                
                # Agregar a datos_tipo con formato esperado por el visualizador
                if aeronave_idx not in self.datos_tipo:
                    self.datos_tipo[aeronave_idx] = {}
                
                self.datos_tipo[aeronave_idx][parametro] = entrada_celda
                
            except Exception as e:
                print(f"  ⚠️ Error procesando {clave}: {e}")
                continue
        
        print(f"  📋 Generados datos_tipo para {len(self.datos_tipo)} aeronaves")
        
    def cargar_desde_excel(self, ruta_datos: str) -> pd.DataFrame:
        """
        DEPRECATED: Usar cargar_desde_bucle_imputacion() para integración con imputation_loop.py
        
        Esta función está mantenida solo para desarrollo/testing.
        """
        print("⚠️ MODO DESARROLLO - Cargando desde Excel")
        print("💡 Use cargar_desde_bucle_imputacion() para integración con el flujo principal.")
        
        # ... resto del código original ...
        self.modo_carga = "desarrollo"
        
        try:
            # Implementación original mantenida para compatibilidad
            from .Modulos.imputacion_correlacion import cargar_y_validar_datos
            self.df_original = cargar_y_validar_datos(ruta_datos)
            print(f"📊 Datos cargados: {self.df_original.shape}")
            return self._procesar_imputaciones_desarrollo()
            
        except Exception as e:
            print(f"❌ Error cargando desde Excel: {e}")
            return None

✅ Clase AnalizadorModelos definida correctamente


In [15]:
# ================================================================================
# SECCIÓN 3: FUNCIONES DE VISUALIZACIÓN Y ANÁLISIS
# ================================================================================

class VisualizadorModelos:
    """
    Clase para crear las visualizaciones dinámicas del sistema de análisis.
    """
    
    def __init__(self, analizador: AnalizadorModelos):
        self.analizador = analizador
        self.colores_tipos = {
            "linear": "#1f77b4",    # Azul
            "poly": "#ff7f0e",      # Naranja
            "log": "#2ca02c",       # Verde
            "pot": "#d62728",       # Rojo
            "exp": "#9467bd"        # Púrpura
        }
        
    def crear_grafica_comparacion_tipos(self, clave_celda: str, criterio_ranking: str = "Confianza"):
        """
        Crear gráfica izquierda: Comparación entre tipos de modelos.
        MODIFICADA: Funciona con modelos individuales del bucle de imputación.
        """
        info_celda = self.analizador.obtener_info_celda(clave_celda)
        if not info_celda:
            return None
        
        fig = go.Figure()
        
        # Verificar si tenemos modelos del bucle (un solo modelo) o múltiples modelos
        if info_celda.get("metodo_original") == "correlacion_bucle":
            # Modo bucle: un solo modelo por correlación
            return self._crear_grafica_modelo_unico(info_celda, "Modelo de Correlación del Bucle")
        else:
            # Modo desarrollo: múltiples modelos por tipo
            return self._crear_grafica_modelos_multiples(info_celda, criterio_ranking)
    
    def _crear_grafica_modelo_unico(self, info_celda: Dict, titulo: str):
        """
        Crear gráfica para un solo modelo (modo bucle de imputación).
        """
        fig = go.Figure()
        
        # Obtener datos del modelo único
        modelo = info_celda.get("mejor_global")
        if not modelo:
            print("❌ No hay modelo disponible")
            return None
        
        # Obtener datos de entrenamiento
        df_filtrado = info_celda["datos_entrenamiento"]["df_filtrado"]
        objetivo = info_celda["objetivo"]
        aeronave_idx = info_celda["aeronave"]
        
        if df_filtrado.empty:
            print("❌ No hay datos de entrenamiento")
            return None
        
        # Buscar columnas disponibles
        df_train = df_filtrado.dropna(subset=[objetivo]) if objetivo in df_filtrado.columns else df_filtrado
        
        if len(df_train.columns) >= 2:
            # Buscar predictor principal
            predictores = modelo.get("predictores", [])
            if not predictores:
                # Buscar cualquier columna numérica
                numericas = df_train.select_dtypes(include=[np.number]).columns.tolist()
                predictores = [col for col in numericas if col != objetivo][:1]
            
            if predictores:
                predictor_principal = predictores[0]
                
                # Puntos de entrenamiento
                fig.add_trace(go.Scatter(
                    x=df_train[predictor_principal],
                    y=df_train[objetivo],
                    mode='markers',
                    name='Datos Entrenamiento',
                    marker=dict(size=8, color='black', symbol='circle'),
                    hovertemplate=f'<b>Aeronave %{{text}}</b><br>' +
                                f'{predictor_principal}: %{{x}}<br>' +
                                f'{objetivo}: %{{y}}<extra></extra>',
                    text=df_train.index
                ))
                
                # Línea del modelo
                x_range = np.linspace(df_train[predictor_principal].min(), 
                                    df_train[predictor_principal].max(), 100)
                
                y_pred = self._calcular_predicciones_modelo(modelo, x_range, predictor_principal)
                
                fig.add_trace(go.Scatter(
                    x=x_range,
                    y=y_pred,
                    mode='lines',
                    name=f"🏆 {modelo['tipo'].title()} (ELEGIDO)",
                    line=dict(width=4, color='#1f77b4'),
                    hovertemplate=f'<b>Modelo Elegido</b><br>' +
                                f'Tipo: {modelo["tipo"]}<br>' +
                                f'MAPE: {modelo["mape"]:.2f}%<br>' +
                                f'R²: {modelo["r2"]:.3f}<br>' +
                                f'Confianza: {modelo["Confianza"]:.3f}<br>' +
                                f'Predictores: {", ".join(predictores)}<extra></extra>'
                ))
                
                # Marcar punto imputado
                valor_imputado = modelo.get("valor_imputado")
                if valor_imputado is not None and aeronave_idx in df_filtrado.index:
                    valor_predictor = df_filtrado.at[aeronave_idx, predictor_principal]
                    
                    fig.add_trace(go.Scatter(
                        x=[valor_predictor],
                        y=[valor_imputado],
                        mode='markers',
                        name='Valor Imputado',
                        marker=dict(size=12, color='red', symbol='star'),
                        hovertemplate=f'<b>Valor Imputado</b><br>' +
                                    f'Aeronave: {aeronave_idx}<br>' +
                                    f'{predictor_principal}: {valor_predictor}<br>' +
                                    f'{objetivo}: {valor_imputado:.2f}<extra></extra>'
                    ))
        
        # Configurar layout
        fig.update_layout(
            title=f"{titulo}<br><sub>Aeronave {aeronave_idx} - {objetivo}</sub>",
            xaxis_title=predictor_principal if 'predictor_principal' in locals() else "Predictor",
            yaxis_title=objetivo,
            hovermode='closest',
            height=500,
            showlegend=True,
            legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
        )
        
        return fig
    
    def _crear_grafica_modelos_multiples(self, info_celda: Dict, criterio_ranking: str):
        """
        Crear gráfica para múltiples modelos (modo desarrollo).
        """
        fig = go.Figure()
        
        # Obtener datos de entrenamiento
        df_filtrado = info_celda["datos_entrenamiento"]["df_filtrado"]
        objetivo = info_celda["objetivo"]
        aeronave_idx = info_celda["aeronave"]
        
        # Agregar puntos de entrenamiento
        df_train = df_filtrado.dropna(subset=[objetivo])
        if len(df_train.columns) >= 2:  # Al menos objetivo + 1 predictor
            predictor_principal = [col for col in df_train.columns 
                                 if col != objetivo and col != df_train.columns[0]][0]
            
            fig.add_trace(go.Scatter(
                x=df_train[predictor_principal],
                y=df_train[objetivo],
                mode='markers',
                name='Datos Entrenamiento',
                marker=dict(size=8, color='black', symbol='circle'),
                hovertemplate=f'<b>Aeronave %{{text}}</b><br>' +
                            f'{predictor_principal}: %{{x}}<br>' +
                            f'{objetivo}: %{{y}}<extra></extra>',
                text=df_train.index
            ))
            
            # Agregar mejores modelos de cada tipo
            x_range = np.linspace(df_train[predictor_principal].min(), 
                                df_train[predictor_principal].max(), 100)
            
            mejor_global = info_celda["mejor_global"]
            
            for tipo, datos_tipo in info_celda["modelos_por_tipo"].items():
                if datos_tipo["mejor"] is None:
                    continue
                    
                modelo = datos_tipo["mejor"]
                
                # Calcular predicciones
                y_pred = self._calcular_predicciones_modelo(modelo, x_range, predictor_principal)
                
                # Determinar estilo de línea
                if (mejor_global and modelo["tipo"] == mejor_global["tipo"] and 
                    modelo["Confianza_promedio"] == mejor_global["Confianza_promedio"]):
                    # Modelo elegido global
                    line_style = dict(width=4, color=self.colores_tipos[tipo])
                    name = f"🏆 {tipo.title()} (ELEGIDO)"
                    opacity = 1.0
                else:
                    # Mejor del tipo pero no elegido global
                    line_style = dict(width=2, color=self.colores_tipos[tipo], dash='dash')
                    name = f"{tipo.title()}"
                    opacity = 0.7
                
                fig.add_trace(go.Scatter(
                    x=x_range,
                    y=y_pred,
                    mode='lines',
                    name=name,
                    line=line_style,
                    opacity=opacity,
                    hovertemplate=f'<b>{name}</b><br>' +
                                f'MAPE: {modelo["mape"]:.2f}%<br>' +
                                f'R²: {modelo["r2"]:.3f}<br>' +
                                f'Confianza: {modelo["Confianza"]:.3f}<br>' +
                                f'Predictores: {", ".join(modelo["predictores"])}<extra></extra>'
                ))
            
            # Marcar punto imputado si existe
            if aeronave_idx in df_filtrado.index:
                valor_predictor = df_filtrado.at[aeronave_idx, predictor_principal]
                if mejor_global:
                    valor_imputado = self._calcular_predicciones_modelo(
                        mejor_global, [valor_predictor], predictor_principal
                    )[0]
                    
                    fig.add_trace(go.Scatter(
                        x=[valor_predictor],
                        y=[valor_imputado],
                        mode='markers',
                        name='Valor Imputado',
                        marker=dict(size=12, color='red', symbol='star'),
                        hovertemplate=f'<b>Valor Imputado</b><br>' +
                                    f'Aeronave: {aeronave_idx}<br>' +
                                    f'{predictor_principal}: {valor_predictor}<br>' +
                                    f'{objetivo}: {valor_imputado:.2f}<extra></extra>'
                    ))
        
        # Configurar layout
        fig.update_layout(
            title=f"Comparación Entre Tipos de Modelos<br>" +
                  f"<sub>Aeronave {aeronave_idx} - {objetivo}</sub>",
            xaxis_title=predictor_principal if 'predictor_principal' in locals() else "Predictor",
            yaxis_title=objetivo,
            hovermode='closest',
            height=500,
            showlegend=True,
            legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
        )
        
        return fig
    
    def crear_grafica_analisis_intra_tipo(self, clave_celda: str, tipo_seleccionado: str, criterio_ranking: str = "Confianza"):
        """
        Crear gráfica derecha: Análisis intra-tipo.
        """
        info_celda = self.analizador.obtener_info_celda(clave_celda)
        if not info_celda or tipo_seleccionado not in info_celda["modelos_por_tipo"]:
            return None
        
        fig = go.Figure()
        
        # Obtener datos del tipo seleccionado
        datos_tipo = info_celda["modelos_por_tipo"][tipo_seleccionado]
        modelos_tipo = datos_tipo["todos"]
        mejor_tipo = datos_tipo["mejor"]
        
        # Obtener datos de entrenamiento
        df_filtrado = info_celda["datos_entrenamiento"]["df_filtrado"]
        objetivo = info_celda["objetivo"]
        aeronave_idx = info_celda["aeronave"]
        
        # Agregar puntos de entrenamiento
        df_train = df_filtrado.dropna(subset=[objetivo])
        if len(df_train.columns) >= 2:
            predictor_principal = [col for col in df_train.columns 
                                 if col != objetivo and col != df_train.columns[0]][0]
            
            fig.add_trace(go.Scatter(
                x=df_train[predictor_principal],
                y=df_train[objetivo],
                mode='markers',
                name='Datos Entrenamiento',
                marker=dict(size=8, color='black', symbol='circle'),
                hovertemplate=f'<b>Aeronave %{{text}}</b><br>' +
                            f'{predictor_principal}: %{{x}}<br>' +
                            f'{objetivo}: %{{y}}<extra></extra>',
                text=df_train.index
            ))
            
            # Agregar todos los modelos del tipo
            x_range = np.linspace(df_train[predictor_principal].min(), 
                                df_train[predictor_principal].max(), 100)
            
            # Ordenar modelos por criterio seleccionado
            modelos_ordenados = sorted(modelos_tipo, 
                                     key=lambda x: x.get(criterio_ranking, 0), 
                                     reverse=True)
            
            for i, modelo in enumerate(modelos_ordenados):
                # Calcular predicciones
                y_pred = self._calcular_predicciones_modelo(modelo, x_range, predictor_principal)
                
                # Determinar estilo basado en ranking
                if modelo == mejor_tipo:
                    # Mejor del tipo
                    line_style = dict(width=4, color=self.colores_tipos[tipo_seleccionado])
                    name = f"🏆 {modelo['tipo']} ({', '.join(modelo['predictores'])})"
                    opacity = 1.0
                else:
                    # Otros modelos del tipo
                    transparency = max(0.3, 1.0 - (i * 0.15))  # Degradado de transparencia
                    line_style = dict(width=2, color=self.colores_tipos[tipo_seleccionado], 
                                    dash='dot' if i > 2 else 'dash')
                    name = f"{modelo['tipo']} ({', '.join(modelo['predictores'])})"
                    opacity = transparency
                
                fig.add_trace(go.Scatter(
                    x=x_range,
                    y=y_pred,
                    mode='lines',
                    name=name,
                    line=line_style,
                    opacity=opacity,
                    hovertemplate=f'<b>{name}</b><br>' +
                                f'MAPE: {modelo["mape"]:.2f}%<br>' +
                                f'R²: {modelo["r2"]:.3f}<br>' +
                                f'Confianza: {modelo["Confianza"]:.3f}<br>' +
                                f'LOOCV Confianza: {modelo.get("Confianza_LOOCV", 0):.3f}<br>' +
                                f'Ranking: {i+1}<extra></extra>'
                ))
        
        # Configurar layout
        fig.update_layout(
            title=f"Análisis Intra-Tipo: {tipo_seleccionado.title()}<br>" +
                  f"<sub>Ranking por {criterio_ranking}</sub>",
            xaxis_title=predictor_principal if 'predictor_principal' in locals() else "Predictor",
            yaxis_title=objetivo,
            hovermode='closest',
            height=500,
            showlegend=True,
            legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
        )
        
        return fig
    
    def _calcular_predicciones_modelo(self, modelo: Dict, x_values: np.ndarray, predictor: str) -> np.ndarray:
        """
        Calcular predicciones de un modelo para valores X dados.
        CORREGIDO: Usa los coeficientes correctamente según el tipo de modelo.
        """
        try:
            # Obtener información del modelo
            coefs = modelo["coeficientes_originales"]
            intercept = modelo["intercepto_original"]
            tipo = modelo["tipo"]
            
            # CORRECCIÓN: Para modelos con transformaciones, necesitamos aplicar las transformaciones correctas
            if tipo.startswith("linear"):
                # Modelo lineal simple: y = a + bx
                if len(coefs) >= 1:
                    return intercept + coefs[0] * x_values
                else:
                    return np.full_like(x_values, intercept)
                    
            elif tipo.startswith("poly") and len(coefs) >= 2:
                # Modelo polinómico: y = a + bx + cx²
                return intercept + coefs[0] * x_values + coefs[1] * (x_values ** 2)
                
            elif tipo.startswith("log"):
                # Modelo logarítmico: y = a + b*ln(x)
                # Verificar que x > 0 para logaritmo
                x_safe = np.maximum(x_values, 1e-10)
                if len(coefs) >= 1:
                    return intercept + coefs[0] * np.log(x_safe)
                else:
                    return np.full_like(x_values, intercept)
                    
            elif tipo.startswith("pot"):
                # Modelo de potencia: y = a * x^b
                # Verificar que x > 0 para potencia
                x_safe = np.maximum(x_values, 1e-10)
                if len(coefs) >= 1:
                    return intercept * (x_safe ** coefs[0])
                else:
                    return np.full_like(x_values, intercept)
                    
            elif tipo.startswith("exp"):
                # Modelo exponencial: y = a * exp(b*x)
                if len(coefs) >= 1:
                    # Limitar el exponente para evitar overflow
                    exp_arg = np.clip(coefs[0] * x_values, -700, 700)
                    return intercept * np.exp(exp_arg)
                else:
                    return np.full_like(x_values, intercept)
            else:
                return np.full_like(x_values, intercept)
                
        except Exception as e:
            print(f"⚠️ Error calculando predicciones para {modelo['tipo']}: {e}")
            # Devolver valores promedio como fallback
            return np.full_like(x_values, intercept if 'intercept' in locals() else 0)

print("✅ Clase VisualizadorModelos definida correctamente")

✅ Clase VisualizadorModelos definida correctamente


In [16]:
# ================================================================================
# SECCIÓN 4: INTERFAZ INTERACTIVA CON WIDGETS
# ================================================================================

class InterfazInteractiva:
    """
    Clase para crear y manejar la interfaz interactiva del sistema.
    """
    
    def __init__(self, analizador: AnalizadorModelos):
        self.analizador = analizador
        self.visualizador = VisualizadorModelos(analizador)
        
        # Widgets principales
        self.selector_celda = None
        self.selector_tipo = None
        self.selector_criterio = None
        self.output_graficas = widgets.Output()
        self.output_metricas = widgets.Output()
        self.output_residuos = widgets.Output()
        self.output_recomendaciones = widgets.Output()
        
        # Estado actual
        self.celda_actual = None
        self.tipo_actual = "linear"
        self.criterio_actual = "Confianza"
        
        # Modos de visualización
        self.mostrar_comparacion = True
        self.mostrar_intra_tipo = True
        
    def crear_controles(self):
        """Crear todos los widgets de control."""
        celdas_disponibles = self.analizador.obtener_celdas_disponibles()
        
        if not celdas_disponibles:
            print("❌ No hay celdas con modelos disponibles. Ejecute la imputación primero.")
            return None
        
        # Selector de celda (aeronave + parámetro)
        opciones_celda = []
        for clave in celdas_disponibles:
            info = self.analizador.obtener_info_celda(clave)
            if info:
                etiqueta = f"Aeronave {info['aeronave']} → {info['objetivo']}"
                opciones_celda.append((etiqueta, clave))
        
        self.selector_celda = widgets.Dropdown(
            options=opciones_celda,
            value=opciones_celda[0][1] if opciones_celda else None,
            description='Celda:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Selector de tipo de modelo
        self.selector_tipo = widgets.Dropdown(
            options=[
                ('Lineal', 'linear'),
                ('Polinómico', 'poly'),
                ('Logarítmico', 'log'),
                ('Potencia', 'pot'),
                ('Exponencial', 'exp')
            ],
            value='linear',
            description='Tipo Modelo:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
        
        # Selector de criterio de ranking
        self.selector_criterio = widgets.Dropdown(
            options=[
                ('Confianza Promedio', 'Confianza_promedio'),
                ('Confianza Simple', 'Confianza'),
                ('MAPE (menor mejor)', 'mape'),
                ('R² (mayor mejor)', 'r2'),
                ('Confianza LOOCV', 'Confianza_LOOCV')
            ],
            value='Confianza_promedio',
            description='Criterio:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
        
        # Configurar callbacks
        self.selector_celda.observe(self._on_change_celda, names='value')
        self.selector_tipo.observe(self._on_change_tipo, names='value')
        self.selector_criterio.observe(self._on_change_criterio, names='value')
        
        # Botones adicionales
        btn_actualizar = widgets.Button(
            description='🔄 Actualizar',
            button_style='info',
            layout=widgets.Layout(width='120px')
        )
        btn_exportar = widgets.Button(
            description='📊 Exportar',
            button_style='success',
            layout=widgets.Layout(width='120px')
        )
        btn_ayuda = widgets.Button(
            description='❓ Ayuda',
            button_style='warning',
            layout=widgets.Layout(width='120px')
        )
        
        # NUEVOS: Botones de control de modo de visualización
        self.btn_comparacion = widgets.ToggleButton(
            value=True,
            description='📊 Comparación',
            disabled=False,
            button_style='success',
            tooltip='Mostrar comparación entre tipos',
            layout=widgets.Layout(width='130px')
        )
        
        self.btn_intra_tipo = widgets.ToggleButton(
            value=True,
            description='🔍 Intra-Tipo',
            disabled=False,
            button_style='success',
            tooltip='Mostrar análisis intra-tipo',
            layout=widgets.Layout(width='130px')
        )
        
        btn_actualizar.on_click(self._on_click_actualizar)
        btn_exportar.on_click(self._on_click_exportar)
        btn_ayuda.on_click(self._on_click_ayuda)
        
        # Callbacks para botones de modo
        self.btn_comparacion.observe(self._on_change_modo_comparacion, names='value')
        self.btn_intra_tipo.observe(self._on_change_modo_intra_tipo, names='value')
        
        # Layout de controles principales
        controles_principales = widgets.HBox([
            self.selector_celda,
            self.selector_tipo,
            self.selector_criterio,
            btn_actualizar,
            btn_exportar,
            btn_ayuda
        ])
        
        # Layout de controles de modo
        controles_modo = widgets.HBox([
            widgets.HTML(value="<b>Modo de Visualización:</b>"),
            self.btn_comparacion,
            self.btn_intra_tipo
        ])
        
        # Combinar todos los controles
        controles = widgets.VBox([
            controles_principales,
            controles_modo
        ])
        
        return controles
    
    def crear_interfaz_completa(self):
        """Crear la interfaz completa del sistema."""
        controles = self.crear_controles()
        if controles is None:
            return None
        
        # Título principal
        titulo = widgets.HTML(
            value="""
            <div style='text-align: center; background: linear-gradient(90deg, #1f77b4, #ff7f0e); 
                        padding: 10px; border-radius: 10px; margin-bottom: 10px;'>
                <h2 style='color: white; margin: 0;'>
                    🎯 Sistema de Análisis Visual Dinámico para Imputación por Correlación
                </h2>
            </div>
            """
        )
        
        # Panel de gráficas (lado a lado)
        panel_graficas = widgets.HBox([
            self.output_graficas
        ], layout=widgets.Layout(width='100%'))
        
        # Panel de métricas y análisis
        tabs_analisis = widgets.Tab()
        tabs_analisis.children = [
            self.output_metricas,
            self.output_residuos,
            self.output_recomendaciones
        ]
        tabs_analisis.set_title(0, '📊 Métricas Comparativas')
        tabs_analisis.set_title(1, '📈 Análisis de Residuos')
        tabs_analisis.set_title(2, '💡 Recomendaciones')
        
        # Layout completo
        interfaz_completa = widgets.VBox([
            titulo,
            controles,
            panel_graficas,
            tabs_analisis
        ])
        
        # Inicializar con la primera celda
        if self.selector_celda.options:
            self.celda_actual = self.selector_celda.value
            self.actualizar_visualizaciones()
        
        return interfaz_completa
    
    def _on_change_celda(self, change):
        """Callback para cambio de celda."""
        self.celda_actual = change['new']
        self.actualizar_visualizaciones()
    
    def _on_change_tipo(self, change):
        """Callback para cambio de tipo."""
        self.tipo_actual = change['new']
        self.actualizar_visualizaciones()
    
    def _on_change_criterio(self, change):
        """Callback para cambio de criterio."""
        self.criterio_actual = change['new']
        self.actualizar_visualizaciones()
    
    def _on_change_modo_comparacion(self, change):
        """Callback para cambio de modo comparación."""
        self.mostrar_comparacion = change['new']
        self.actualizar_visualizaciones()
    
    def _on_change_modo_intra_tipo(self, change):
        """Callback para cambio de modo intra-tipo."""
        self.mostrar_intra_tipo = change['new']
        self.actualizar_visualizaciones()
    
    def _on_click_actualizar(self, btn):
        """Callback para botón actualizar."""
        self.actualizar_visualizaciones()
    
    def _on_click_exportar(self, btn):
        """Callback para botón exportar."""
        self.exportar_analisis()
    
    def _on_click_ayuda(self, btn):
        """Callback para botón ayuda."""
        self.mostrar_ayuda()
    
    def actualizar_visualizaciones(self):
        """Actualizar todas las visualizaciones."""
        if not self.celda_actual:
            return
        
        # Limpiar outputs
        self.output_graficas.clear_output()
        self.output_metricas.clear_output()
        self.output_residuos.clear_output()
        self.output_recomendaciones.clear_output()
        
        # Actualizar gráficas principales
        with self.output_graficas:
            self._actualizar_graficas_principales()
        
        # Actualizar métricas
        with self.output_metricas:
            self._actualizar_panel_metricas()
        
        # Actualizar análisis de residuos
        with self.output_residuos:
            self._actualizar_analisis_residuos()
        
        # Actualizar recomendaciones
        with self.output_recomendaciones:
            self._actualizar_recomendaciones()
    
    def _actualizar_graficas_principales(self):
        """Actualizar las gráficas principales con control de modo unificado."""
        from plotly.subplots import make_subplots
        
        # Obtener configuración actual de los botones
        modo_comparacion = getattr(self, 'mostrar_comparacion', True)
        modo_intra_tipo = getattr(self, 'mostrar_intra_tipo', True)
        
        if modo_comparacion and modo_intra_tipo:
            # Modo dual: mostrar ambas gráficas lado a lado
            fig = make_subplots(
                rows=1, cols=2,
                subplot_titles=("Comparación Entre Tipos", f"Análisis Intra-Tipo: {self.tipo_actual.title()}"),
                horizontal_spacing=0.1
            )
            
            # Gráfica izquierda: Comparación entre tipos
            fig_izq = self.visualizador.crear_grafica_comparacion_tipos(
                self.celda_actual, self.criterio_actual
            )
            if fig_izq:
                for trace in fig_izq.data:
                    fig.add_trace(trace, row=1, col=1)
            
            # Gráfica derecha: Análisis intra-tipo
            fig_der = self.visualizador.crear_grafica_analisis_intra_tipo(
                self.celda_actual, self.tipo_actual, self.criterio_actual
            )
            if fig_der:
                for trace in fig_der.data:
                    fig.add_trace(trace, row=1, col=2)
            
            # Configurar layout dual
            info_celda = self.analizador.obtener_info_celda(self.celda_actual)
            if info_celda:
                fig.update_layout(
                    title_text=f"Análisis Dual - Aeronave {info_celda['aeronave']} → {info_celda['objetivo']}",
                    height=600,
                    showlegend=False
                )
        
        elif modo_comparacion:
            # Modo solo comparación entre tipos
            fig = self.visualizador.crear_grafica_comparacion_tipos(
                self.celda_actual, self.criterio_actual
            )
            if fig:
                fig.update_layout(height=600, title_font_size=16)
        
        elif modo_intra_tipo:
            # Modo solo análisis intra-tipo
            fig = self.visualizador.crear_grafica_analisis_intra_tipo(
                self.celda_actual, self.tipo_actual, self.criterio_actual
            )
            if fig:
                fig.update_layout(height=600, title_font_size=16)
        
        else:
            # Sin gráficas
            print("ℹ️  Seleccione al menos un modo de visualización")
            return
        
        if fig:
            fig.show()
    
    def _actualizar_panel_metricas(self):
        """Actualizar el panel de métricas comparativas."""
        info_celda = self.analizador.obtener_info_celda(self.celda_actual)
        if not info_celda:
            print("❌ No hay información para esta celda")
            return
        
        # Crear tabla de métricas por tipo
        print("📊 MÉTRICAS COMPARATIVAS POR TIPO DE MODELO")
        print("=" * 60)
        
        datos_tabla = []
        for tipo, datos_tipo in info_celda["modelos_por_tipo"].items():
            if datos_tipo["mejor"] is None:
                continue
                
            modelo = datos_tipo["mejor"]
            datos_tabla.append({
                "Tipo": tipo.title(),
                "MAPE (%)": f"{modelo['mape']:.2f}",
                "R²": f"{modelo['r2']:.3f}",
                "Confianza": f"{modelo['Confianza']:.3f}",
                "LOOCV": f"{modelo.get('Confianza_LOOCV', 0):.3f}",
                "Promedio": f"{modelo['Confianza_promedio']:.3f}",
                "Predictores": ", ".join(modelo['predictores'])
            })
        
        df_metricas = pd.DataFrame(datos_tabla)
        print(df_metricas.to_string(index=False))
        
        # Resaltar el mejor modelo global
        mejor_global = info_celda["mejor_global"]
        if mejor_global:
            print(f"\n🏆 MODELO ELEGIDO GLOBALMENTE:")
            print(f"   Tipo: {mejor_global['tipo']}")
            print(f"   MAPE: {mejor_global['mape']:.2f}%")
            print(f"   R²: {mejor_global['r2']:.3f}")
            print(f"   Confianza Promedio: {mejor_global['Confianza_promedio']:.3f}")
            print(f"   Predictores: {', '.join(mejor_global['predictores'])}")
        
        # Estadísticas del tipo seleccionado
        if self.tipo_actual in info_celda["modelos_por_tipo"]:
            datos_tipo = info_celda["modelos_por_tipo"][self.tipo_actual]
            print(f"\n📈 ESTADÍSTICAS DEL TIPO {self.tipo_actual.upper()}:")
            print(f"   Total de modelos: {len(datos_tipo['todos'])}")
            
            # Ranking de modelos del tipo
            modelos_ordenados = sorted(datos_tipo["todos"], 
                                     key=lambda x: x.get(self.criterio_actual, 0), 
                                     reverse=True)
            
            print(f"   Top 3 por {self.criterio_actual}:")
            for i, modelo in enumerate(modelos_ordenados[:3]):
                print(f"      {i+1}. {modelo['tipo']} - {self.criterio_actual}: {modelo.get(self.criterio_actual, 0):.3f}")
    
    def _actualizar_analisis_residuos(self):
        """Actualizar el análisis de residuos."""
        info_celda = self.analizador.obtener_info_celda(self.celda_actual)
        if not info_celda:
            return
        
        print("📈 ANÁLISIS DE RESIDUOS Y OUTLIERS")
        print("=" * 40)
        
        # Analizar el mejor modelo global
        mejor_global = info_celda["mejor_global"]
        if not mejor_global:
            print("❌ No hay modelo global válido para analizar")
            return
        
        # Obtener datos de entrenamiento
        df_filtrado = info_celda["datos_entrenamiento"]["df_filtrado"]
        objetivo = info_celda["objetivo"]
        df_train = df_filtrado.dropna(subset=[objetivo])
        
        # Calcular residuos
        residuos = []
        valores_reales = []
        indices_aeronaves = []
        
        for idx in df_train.index:
            # Preparar datos del modelo
            fila_input = {}
            for pred in mejor_global["predictores"]:
                if pred in df_train.columns:
                    fila_input[pred] = df_train.at[idx, pred]
            
            # Calcular predicción (simplificada para un predictor)
            if len(mejor_global["predictores"]) == 1:
                pred_name = mejor_global["predictores"][0]
                valor_real = df_train.at[idx, objetivo]
                valor_pred = self.visualizador._calcular_predicciones_modelo(
                    mejor_global, [df_train.at[idx, pred_name]], pred_name
                )[0]
                
                residuo = valor_real - valor_pred
                residuos.append(residuo)
                valores_reales.append(valor_real)
                indices_aeronaves.append(idx)
        
        if residuos:
            residuos = np.array(residuos)
            
            # Estadísticas de residuos
            print(f"📊 Estadísticas de Residuos ({mejor_global['tipo']}):")
            print(f"   Media: {np.mean(residuos):.4f}")
            print(f"   Desviación estándar: {np.std(residuos):.4f}")
            print(f"   Rango: [{np.min(residuos):.4f}, {np.max(residuos):.4f}]")
            
            # Detectar outliers (usando regla 2σ)
            umbral = 2 * np.std(residuos)
            outliers = np.abs(residuos) > umbral
            
            if np.any(outliers):
                print(f"\n⚠️  OUTLIERS DETECTADOS ({np.sum(outliers)} de {len(residuos)}):")
                for i, (idx, residuo) in enumerate(zip(indices_aeronaves, residuos)):
                    if outliers[i]:
                        print(f"   Aeronave {idx}: Residuo = {residuo:.4f}")
            else:
                print("\n✅ No se detectaron outliers significativos")
            
            # Test de normalidad (Shapiro-Wilk si n < 50)
            if len(residuos) < 50:
                from scipy.stats import shapiro
                stat, p_value = shapiro(residuos)
                print(f"\n📊 Test de Normalidad (Shapiro-Wilk):")
                print(f"   Estadístico: {stat:.4f}")
                print(f"   p-value: {p_value:.4f}")
                if p_value > 0.05:
                    print("   ✅ Los residuos parecen seguir una distribución normal")
                else:
                    print("   ⚠️  Los residuos NO siguen una distribución normal")
    
    def _actualizar_recomendaciones(self):
        """Actualizar las recomendaciones automáticas."""
        info_celda = self.analizador.obtener_info_celda(self.celda_actual)
        if not info_celda:
            return
        
        print("💡 RECOMENDACIONES AUTOMÁTICAS")
        print("=" * 35)
        
        mejor_global = info_celda["mejor_global"]
        if not mejor_global:
            print("❌ No hay modelo válido. Recomendaciones:")
            print("   • Revisar la calidad de los datos")
            print("   • Considerar más predictores")
            print("   • Verificar outliers en los datos")
            return
        
        mape = mejor_global["mape"]
        r2 = mejor_global["r2"]
        confianza = mejor_global["Confianza_promedio"]
        
        recomendaciones = []
        
        # Evaluación de MAPE
        if mape <= 3.0:
            recomendaciones.append("✅ EXCELENTE: MAPE muy bajo, modelo muy confiable")
        elif mape <= 5.0:
            recomendaciones.append("✅ BUENO: MAPE aceptable, modelo confiable")
        elif mape <= 7.5:
            recomendaciones.append("⚠️  ACEPTABLE: MAPE en límite, revisar si es suficiente")
        else:
            recomendaciones.append("❌ PROBLEMÁTICO: MAPE alto, considerar otros enfoques")
        
        # Evaluación de R²
        if r2 >= 0.9:
            recomendaciones.append("✅ EXCELENTE: R² muy alto, excelente ajuste")
        elif r2 >= 0.8:
            recomendaciones.append("✅ BUENO: R² alto, buen ajuste")
        elif r2 >= 0.6:
            recomendaciones.append("⚠️  ACEPTABLE: R² mínimo aceptable")
        else:
            recomendaciones.append("❌ PROBLEMÁTICO: R² bajo, ajuste insuficiente")
        
        # Evaluación de confianza
        if confianza >= 0.8:
            recomendaciones.append("✅ ALTA CONFIANZA: Modelo muy confiable")
        elif confianza >= 0.6:
            recomendaciones.append("⚠️  CONFIANZA MEDIA: Usar con precaución")
        else:
            recomendaciones.append("❌ BAJA CONFIANZA: No recomendable para uso")
        
        # Análisis comparativo entre tipos
        tipos_disponibles = list(info_celda["modelos_por_tipo"].keys())
        if len(tipos_disponibles) > 1:
            mejores_por_tipo = []
            for tipo, datos in info_celda["modelos_por_tipo"].items():
                if datos["mejor"]:
                    mejores_por_tipo.append((tipo, datos["mejor"]["Confianza_promedio"]))
            
            mejores_por_tipo.sort(key=lambda x: x[1], reverse=True)
            
            if len(mejores_por_tipo) >= 2:
                mejor_tipo, mejor_conf = mejores_por_tipo[0]
                segundo_tipo, segunda_conf = mejores_por_tipo[1]
                
                if mejor_conf - segunda_conf < 0.05:
                    recomendaciones.append(f"⚠️  COMPETENCIA: {mejor_tipo} y {segundo_tipo} muy similares")
                else:
                    recomendaciones.append(f"✅ CLARO: {mejor_tipo} claramente superior")
        
        # Recomendaciones específicas
        n_predictores = len(mejor_global["predictores"])
        if n_predictores == 1:
            recomendaciones.append("💡 SUGERENCIA: Considerar modelos con más predictores")
        elif n_predictores > 3:
            recomendaciones.append("⚠️  ADVERTENCIA: Modelo complejo, riesgo de sobreajuste")
        
        # Mostrar todas las recomendaciones
        for rec in recomendaciones:
            print(f"   {rec}")
        
        # Acciones sugeridas
        print(f"\n🎯 ACCIONES SUGERIDAS:")
        if mape > 7.5 or r2 < 0.6:
            print("   • Revisar datos de entrenamiento")
            print("   • Explorar otros tipos de modelos")
            print("   • Verificar outliers")
        
        if confianza < 0.7:
            print("   • Aumentar datos de entrenamiento")
            print("   • Considerar validación cruzada más robusta")
        
        print("   • Validar con datos independientes antes de usar")
        print("   • Monitorear rendimiento en producción")
    
    def exportar_analisis(self):
        """Exportar análisis completo a archivo."""
        print("📊 Función de exportación - En desarrollo")
        print("   Próximamente: Exportación a Excel, PDF y HTML")
    
    def mostrar_ayuda(self):
        """Mostrar ayuda del sistema."""
        with self.output_recomendaciones:
            clear_output()
            print("❓ AYUDA DEL SISTEMA")
            print("=" * 25)
            print("📖 GUÍA DE USO:")
            print("   1. Seleccione una celda (aeronave + parámetro)")
            print("   2. Seleccione un tipo de modelo para análisis detallado")
            print("   3. Elija criterio de ranking")
            print("   4. Revise las métricas y recomendaciones")
            print("\n🎯 INTERPRETACIÓN:")
            print("   • MAPE < 5%: Excelente")
            print("   • R² > 0.8: Muy buen ajuste")
            print("   • Confianza > 0.7: Modelo confiable")
            print("\n🔍 FUNCIONALIDADES:")
            print("   • Gráfica izquierda: Comparación entre tipos")
            print("   • Gráfica derecha: Análisis dentro del tipo")
            print("   • Pestañas: Métricas, residuos, recomendaciones")

print("✅ Clase InterfazInteractiva definida correctamente")

✅ Clase InterfazInteractiva definida correctamente


In [ ]:
# ================================================================================
# SECCIÓN 5: INICIALIZACIÓN Y CONFIGURACIÓN DEL SISTEMA
# ================================================================================

# Configuración global
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
np.set_printoptions(precision=4, suppress=True)

print("🚀 INICIANDO SISTEMA DE ANÁLISIS VISUAL DINÁMICO")
print("=" * 50)

# Crear instancia del analizador
analizador = AnalizadorModelos()

# OPCIÓN 1: Cargar desde diccionarios del flujo principal (RECOMENDADO)
# Si tienes los diccionarios disponibles, úsalos así:
# datos = analizador.cargar_desde_diccionarios(df_original, diccionarios_modelos)

# OPCIÓN 2: Cargar desde archivo Excel (para desarrollo/testing)
print("🔄 Modo de carga:")
print("   📁 OPCIÓN A: Desde archivo Excel (desarrollo)")
print("   📊 OPCIÓN B: Desde diccionarios del flujo principal (producción)")

# Detectar si hay diccionarios disponibles en el entorno
if 'diccionarios_modelos_globales' in globals() or 'df_resultado_imputacion' in globals():
    print("   ✅ Detectados diccionarios del flujo principal")
    print("   🔄 Cargando desde diccionarios...")
    
    # Cargar desde diccionarios (implementar esta función)
    try:
        datos = analizador.cargar_desde_diccionarios(
            globals().get('df_original', None),
            globals().get('diccionarios_modelos_globales', {}),
            globals().get('df_resultado_imputacion', None)
        )
        print("   ✅ Datos cargados desde diccionarios del flujo principal")
    except Exception as e:
        print(f"   ❌ Error cargando desde diccionarios: {e}")
        print("   🔄 Fallback a carga desde Excel...")
        datos = None
else:
    print("   ℹ️  No se detectaron diccionarios, usando archivo Excel")
    datos = None

# Fallback: Cargar desde Excel si no hay diccionarios
if datos is None:
    RUTA_DATOS = "Data/Datos_aeronaves.xlsx"
    print(f"📂 Cargando datos desde: {RUTA_DATOS}")
    
    # Verificar si el archivo existe
    import os
    if not os.path.exists(RUTA_DATOS):
        print(f"❌ Error: No se encuentra el archivo {RUTA_DATOS}")
        print("💡 Opciones disponibles:")
        for archivo in os.listdir("Data"):
            if archivo.endswith(('.xlsx', '.csv')):
                print(f"   - Data/{archivo}")
        RUTA_DATOS = "Data/Datos_aeronaves.xlsx"  # Fallback
        print(f"🔄 Usando archivo alternativo: {RUTA_DATOS}")
    
    # Cargar datos
    datos = analizador.cargar_datos(RUTA_DATOS)

if datos is not None:
    print(f"✅ Datos cargados exitosamente")
    print(f"   - Filas: {datos.shape[0]}")
    print(f"   - Columnas: {datos.shape[1]}")
    print(f"   - Valores faltantes por columna:")
    
    faltantes = datos.isnull().sum()
    for col, count in faltantes[faltantes > 0].items():
        print(f"     • {col}: {count} valores ({count/len(datos)*100:.1f}%)")
else:
    print("❌ Error: No se pudieron cargar los datos")

print("\n" + "="*50)

🚀 INICIANDO SISTEMA DE ANÁLISIS VISUAL DINÁMICO
📂 Cargando datos desde: Data/Datos_aeronaves.xlsx
✅ Datos cargados: (10, 16)
✅ Datos cargados exitosamente
   - Filas: 10
   - Columnas: 16
   - Valores faltantes por columna:
     • payload: 2 valores (20.0%)
     • Potencia HP: 1 valores (10.0%)
     • envergadura: 1 valores (10.0%)
     • Alcance de la aeronave: 1 valores (10.0%)
     • Velocidad a la que se realiza el crucero (KTAS): 1 valores (10.0%)



In [8]:
# ================================================================================
# SECCIÓN 6: EJECUCIÓN DE IMPUTACIÓN Y RECOPILACIÓN DE MODELOS
# ================================================================================

if datos is not None:
    print("🔄 EJECUTANDO IMPUTACIÓN COMPLETA Y RECOPILANDO MODELOS...")
    print("⏳ Este proceso puede tomar varios minutos dependiendo del tamaño de los datos")
    print()
    
    try:
        # Ejecutar imputación completa
        import time
        inicio = time.time()
        
        df_resultado, reporte = analizador.ejecutar_imputacion_completa()
        
        fin = time.time()
        tiempo_total = fin - inicio
        
        print(f"\n✅ PROCESO COMPLETADO EN {tiempo_total:.1f} SEGUNDOS")
        print("=" * 50)
        
        # Mostrar estadísticas del proceso
        celdas_procesadas = len(analizador.modelos_por_celda)
        print(f"📊 ESTADÍSTICAS DEL PROCESO:")
        print(f"   • Celdas analizadas: {celdas_procesadas}")
        
        if celdas_procesadas > 0:
            # Estadísticas por tipo de modelo
            tipos_conteo = {}
            modelos_totales = 0
            
            for clave, info in analizador.modelos_por_celda.items():
                for tipo, datos_tipo in info["modelos_por_tipo"].items():
                    if tipo not in tipos_conteo:
                        tipos_conteo[tipo] = 0
                    tipos_conteo[tipo] += len(datos_tipo["todos"])
                    modelos_totales += len(datos_tipo["todos"])
            
            print(f"   • Total de modelos entrenados: {modelos_totales}")
            print(f"   • Distribución por tipo:")
            for tipo, count in tipos_conteo.items():
                print(f"     - {tipo.title()}: {count} modelos ({count/modelos_totales*100:.1f}%)")
            
            # Estadísticas de éxito
            celdas_con_modelo_valido = sum(1 for info in analizador.modelos_por_celda.values() 
                                         if info["mejor_global"] is not None)
            print(f"   • Celdas con modelo válido: {celdas_con_modelo_valido}/{celdas_procesadas} ({celdas_con_modelo_valido/celdas_procesadas*100:.1f}%)")
        
        print(f"\n🎯 ¡SISTEMA LISTO PARA ANÁLISIS VISUAL!")
        
    except Exception as e:
        print(f"❌ ERROR durante la ejecución: {e}")
        print("💡 Posibles soluciones:")
        print("   • Verificar la integridad de los datos")
        print("   • Revisar las dependencias del módulo imputacion_correlacion")
        print("   • Intentar con un subset más pequeño de datos")
        
else:
    print("❌ No se puede proceder sin datos válidos")

print("\n" + "="*50)

🔄 EJECUTANDO IMPUTACIÓN COMPLETA Y RECOPILANDO MODELOS...
⏳ Este proceso puede tomar varios minutos dependiendo del tamaño de los datos

🔄 Ejecutando imputación completa...
🔄 Recopilando todos los modelos entrenados...
✅ Recopilados modelos para 6 celdas

✅ PROCESO COMPLETADO EN 4.7 SEGUNDOS
📊 ESTADÍSTICAS DEL PROCESO:
   • Celdas analizadas: 6
   • Total de modelos entrenados: 240
   • Distribución por tipo:
     - Linear: 96 modelos (40.0%)
     - Poly: 36 modelos (15.0%)
     - Log: 36 modelos (15.0%)
     - Pot: 36 modelos (15.0%)
     - Exp: 36 modelos (15.0%)
   • Celdas con modelo válido: 6/6 (100.0%)

🎯 ¡SISTEMA LISTO PARA ANÁLISIS VISUAL!



In [9]:
# ================================================================================
# SECCIÓN 7: INTERFAZ PRINCIPAL INTERACTIVA
# ================================================================================

# Verificar que tenemos modelos para analizar
if 'analizador' in locals() and len(analizador.modelos_por_celda) > 0:
    print("🎮 CREANDO INTERFAZ INTERACTIVA...")
    
    # Crear interfaz
    interfaz = InterfazInteractiva(analizador)
    interfaz_completa = interfaz.crear_interfaz_completa()
    
    if interfaz_completa is not None:
        print("✅ Interfaz creada exitosamente")
        print("\n🎯 INSTRUCCIONES DE USO:")
        print("   1. Use los controles superiores para seleccionar:")
        print("      • Celda (aeronave + parámetro a analizar)")
        print("      • Tipo de modelo para análisis detallado")
        print("      • Criterio de ranking de modelos")
        print("   2. Las gráficas se actualizarán automáticamente")
        print("   3. Revise las pestañas inferiores para:")
        print("      • Métricas comparativas")
        print("      • Análisis de residuos")
        print("      • Recomendaciones automáticas")
        print("\n" + "="*50)
        print("🚀 INICIANDO INTERFAZ INTERACTIVA:")
        print("="*50)
        
        # Mostrar la interfaz
        display(interfaz_completa)
        
    else:
        print("❌ Error: No se pudo crear la interfaz")
        print("💡 Verifique que hay datos y modelos disponibles")
        
else:
    print("❌ Error: No hay modelos disponibles para análisis")
    print("💡 Ejecute primero las celdas de carga de datos e imputación")

🎮 CREANDO INTERFAZ INTERACTIVA...


✅ Interfaz creada exitosamente

🎯 INSTRUCCIONES DE USO:
   1. Use los controles superiores para seleccionar:
      • Celda (aeronave + parámetro a analizar)
      • Tipo de modelo para análisis detallado
      • Criterio de ranking de modelos
   2. Las gráficas se actualizarán automáticamente
   3. Revise las pestañas inferiores para:
      • Métricas comparativas
      • Análisis de residuos
      • Recomendaciones automáticas

🚀 INICIANDO INTERFAZ INTERACTIVA:


In [10]:
# ================================================================================
# SECCIÓN 8: FUNCIONALIDADES AVANZADAS
# ================================================================================

def crear_analisis_3d(clave_celda: str, tipo_modelo: str = "linear"):
    """
    Crear visualización 3D para modelos con 2 predictores.
    """
    if 'analizador' not in locals() and 'analizador' not in globals():
        print("❌ Error: Analizador no disponible")
        return None
    
    # Obtener el analizador del alcance global
    analizador_global = globals().get('analizador') or locals().get('analizador')
    
    info_celda = analizador_global.obtener_info_celda(clave_celda)
    if not info_celda or tipo_modelo not in info_celda["modelos_por_tipo"]:
        print(f"❌ No hay datos para {clave_celda} - {tipo_modelo}")
        return None
    
    # Buscar modelos con 2 predictores
    modelos_2d = [m for m in info_celda["modelos_por_tipo"][tipo_modelo]["todos"] 
                  if len(m["predictores"]) == 2]
    
    if not modelos_2d:
        print(f"❌ No hay modelos de 2 predictores para {tipo_modelo}")
        return None
    
    # Tomar el mejor modelo 2D
    mejor_2d = max(modelos_2d, key=lambda x: x["Confianza_promedio"])
    
    print(f"📊 Creando visualización 3D para:")
    print(f"   Modelo: {mejor_2d['tipo']}")
    print(f"   Predictores: {mejor_2d['predictores']}")
    print(f"   MAPE: {mejor_2d['mape']:.2f}%")
    print(f"   R²: {mejor_2d['r2']:.3f}")
    
    # Obtener datos
    df_filtrado = info_celda["datos_entrenamiento"]["df_filtrado"]
    objetivo = info_celda["objetivo"]
    pred1, pred2 = mejor_2d["predictores"]
    
    df_train = df_filtrado.dropna(subset=[objetivo])
    
    # Crear malla 3D
    x_range = np.linspace(df_train[pred1].min(), df_train[pred1].max(), 20)
    y_range = np.linspace(df_train[pred2].min(), df_train[pred2].max(), 20)
    X, Y = np.meshgrid(x_range, y_range)
    
    # Calcular predicciones 3D (simplificado para modelos lineales)
    if mejor_2d["tipo"].startswith("linear"):
        coefs = mejor_2d["coeficientes_originales"]
        intercept = mejor_2d["intercepto_original"]
        Z = intercept + coefs[0] * X + coefs[1] * Y
    else:
        # Para otros tipos, usar predicción punto a punto
        Z = np.zeros_like(X)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                # Simplificación: usar solo el primer predictor
                Z[i, j] = intercept + coefs[0] * X[i, j]
    
    # Crear figura 3D
    fig = go.Figure()
    
    # Superficie del modelo
    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        opacity=0.7,
        name=f'Superficie {mejor_2d["tipo"]}'
    ))
    
    # Puntos de entrenamiento
    fig.add_trace(go.Scatter3d(
        x=df_train[pred1],
        y=df_train[pred2],
        z=df_train[objetivo],
        mode='markers',
        marker=dict(size=8, color='red'),
        name='Datos Entrenamiento'
    ))
    
    # Configurar layout
    fig.update_layout(
        title=f"Visualización 3D - {mejor_2d['tipo']}<br>" +
              f"<sub>{pred1} vs {pred2} → {objetivo}</sub>",
        scene=dict(
            xaxis_title=pred1,
            yaxis_title=pred2,
            zaxis_title=objetivo
        ),
        height=600
    )
    
    fig.show()
    return fig

def exportar_reporte_completo(ruta_salida: str = "analisis_completo.xlsx"):
    """
    Exportar reporte completo del análisis a Excel.
    """
    if 'analizador' not in globals():
        print("❌ Error: Analizador no disponible")
        return
    
    print(f"📊 Exportando reporte completo a: {ruta_salida}")
    
    try:
        with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
            # Hoja 1: Resumen general
            datos_resumen = []
            for clave, info in analizador.modelos_por_celda.items():
                mejor = info["mejor_global"]
                if mejor:
                    datos_resumen.append({
                        "Aeronave": info["aeronave"],
                        "Parametro": info["objetivo"],
                        "Tipo_Elegido": mejor["tipo"],
                        "MAPE": mejor["mape"],
                        "R2": mejor["r2"],
                        "Confianza": mejor["Confianza"],
                        "Confianza_LOOCV": mejor.get("Confianza_LOOCV", 0),
                        "Confianza_Promedio": mejor["Confianza_promedio"],
                        "Predictores": ", ".join(mejor["predictores"]),
                        "Familia_Usada": info["familia_usada"]
                    })
            
            df_resumen = pd.DataFrame(datos_resumen)
            df_resumen.to_excel(writer, sheet_name='Resumen_General', index=False)
            
            # Hoja 2: Métricas por tipo
            datos_tipos = []
            for clave, info in analizador.modelos_por_celda.items():
                for tipo, datos_tipo in info["modelos_por_tipo"].items():
                    if datos_tipo["mejor"]:
                        modelo = datos_tipo["mejor"]
                        datos_tipos.append({
                            "Aeronave": info["aeronave"],
                            "Parametro": info["objetivo"],
                            "Tipo": tipo,
                            "MAPE": modelo["mape"],
                            "R2": modelo["r2"],
                            "Confianza": modelo["Confianza"],
                            "Total_Modelos_Tipo": len(datos_tipo["todos"]),
                            "Predictores": ", ".join(modelo["predictores"])
                        })
            
            df_tipos = pd.DataFrame(datos_tipos)
            df_tipos.to_excel(writer, sheet_name='Metricas_Por_Tipo', index=False)
            
            # Hoja 3: Estadísticas globales
            stats_globales = {
                "Metrica": ["Total_Celdas", "Celdas_Exitosas", "Tasa_Exito", 
                           "MAPE_Promedio", "R2_Promedio", "Confianza_Promedio"],
                "Valor": [
                    len(analizador.modelos_por_celda),
                    len(datos_resumen),
                    len(datos_resumen) / len(analizador.modelos_por_celda) * 100,
                    df_resumen["MAPE"].mean() if not df_resumen.empty else 0,
                    df_resumen["R2"].mean() if not df_resumen.empty else 0,
                    df_resumen["Confianza_Promedio"].mean() if not df_resumen.empty else 0
                ]
            }
            df_stats = pd.DataFrame(stats_globales)
            df_stats.to_excel(writer, sheet_name='Estadisticas_Globales', index=False)
        
        print(f"✅ Reporte exportado exitosamente")
        print(f"📁 Archivo: {ruta_salida}")
        
    except Exception as e:
        print(f"❌ Error exportando: {e}")

def analisis_rapido_celda(aeronave_idx: int, parametro: str):
    """
    Análisis rápido de una celda específica.
    """
    clave_celda = f"aeronave_{aeronave_idx}_parametro_{parametro}"
    
    if 'analizador' not in globals():
        print("❌ Error: Analizador no disponible")
        return
    
    info = analizador.obtener_info_celda(clave_celda)
    if not info:
        print(f"❌ No hay información para Aeronave {aeronave_idx} - {parametro}")
        return
    
    print(f"🔍 ANÁLISIS RÁPIDO: Aeronave {aeronave_idx} → {parametro}")
    print("=" * 50)
    
    mejor = info["mejor_global"]
    if mejor:
        print(f"🏆 MEJOR MODELO:")
        print(f"   Tipo: {mejor['tipo']}")
        print(f"   MAPE: {mejor['mape']:.2f}%")
        print(f"   R²: {mejor['r2']:.3f}")
        print(f"   Confianza: {mejor['Confianza_promedio']:.3f}")
        print(f"   Predictores: {', '.join(mejor['predictores'])}")
        
        # Comparación con otros tipos
        print(f"\n📊 COMPARACIÓN CON OTROS TIPOS:")
        for tipo, datos_tipo in info["modelos_por_tipo"].items():
            if datos_tipo["mejor"] and tipo != mejor["tipo"].split("-")[0]:
                m = datos_tipo["mejor"]
                print(f"   {tipo.title()}: MAPE={m['mape']:.2f}%, R²={m['r2']:.3f}, Conf={m['Confianza_promedio']:.3f}")
    else:
        print("❌ No hay modelo válido para esta celda")
    
    print(f"\n📈 ESTADÍSTICAS:")
    total_modelos = sum(len(datos["todos"]) for datos in info["modelos_por_tipo"].values())
    print(f"   Total modelos entrenados: {total_modelos}")
    print(f"   Tipos disponibles: {len(info['modelos_por_tipo'])}")
    print(f"   Familia de datos usada: {info['familia_usada']}")

# Funciones de utilidad para análisis ad-hoc
def listar_celdas_disponibles():
    """Listar todas las celdas disponibles para análisis."""
    if 'analizador' not in globals():
        print("❌ Error: Analizador no disponible")
        return []
    
    print("📋 CELDAS DISPONIBLES PARA ANÁLISIS:")
    print("=" * 40)
    
    celdas = []
    for clave, info in analizador.modelos_por_celda.items():
        aeronave = info["aeronave"]
        parametro = info["objetivo"]
        tiene_modelo = info["mejor_global"] is not None
        estado = "✅" if tiene_modelo else "❌"
        
        print(f"   {estado} Aeronave {aeronave} → {parametro}")
        celdas.append((aeronave, parametro, tiene_modelo))
    
    return celdas

def comparar_predictores(parametro: str):
    """Comparar qué predictores son más efectivos para un parámetro específico."""
    if 'analizador' not in globals():
        print("❌ Error: Analizador no disponible")
        return
    
    print(f"🔍 ANÁLISIS DE PREDICTORES PARA: {parametro}")
    print("=" * 50)
    
    # Recopilar información de predictores
    uso_predictores = {}
    
    for clave, info in analizador.modelos_por_celda.items():
        if info["objetivo"] == parametro and info["mejor_global"]:
            for pred in info["mejor_global"]["predictores"]:
                if pred not in uso_predictores:
                    uso_predictores[pred] = {
                        "count": 0,
                        "mape_promedio": 0,
                        "r2_promedio": 0,
                        "mapes": [],
                        "r2s": []
                    }
                
                uso_predictores[pred]["count"] += 1
                uso_predictores[pred]["mapes"].append(info["mejor_global"]["mape"])
                uso_predictores[pred]["r2s"].append(info["mejor_global"]["r2"])
    
    # Calcular promedios
    for pred, stats in uso_predictores.items():
        stats["mape_promedio"] = np.mean(stats["mapes"])
        stats["r2_promedio"] = np.mean(stats["r2s"])
    
    # Ordenar por frecuencia de uso
    predictores_ordenados = sorted(uso_predictores.items(), 
                                 key=lambda x: x[1]["count"], 
                                 reverse=True)
    
    print("📊 RANKING DE PREDICTORES:")
    for pred, stats in predictores_ordenados:
        print(f"   {pred}:")
        print(f"     • Usado en: {stats['count']} modelos")
        print(f"     • MAPE promedio: {stats['mape_promedio']:.2f}%")
        print(f"     • R² promedio: {stats['r2_promedio']:.3f}")

print("✅ Funcionalidades avanzadas disponibles:")
print("   • crear_analisis_3d(clave_celda, tipo_modelo)")
print("   • exportar_reporte_completo(ruta_salida)")
print("   • analisis_rapido_celda(aeronave_idx, parametro)")
print("   • listar_celdas_disponibles()")
print("   • comparar_predictores(parametro)")

✅ Funcionalidades avanzadas disponibles:
   • crear_analisis_3d(clave_celda, tipo_modelo)
   • exportar_reporte_completo(ruta_salida)
   • analisis_rapido_celda(aeronave_idx, parametro)
   • listar_celdas_disponibles()
   • comparar_predictores(parametro)


## 🎯 **Ejemplos de Uso del Sistema**

### **Uso Básico de la Interfaz Interactiva**

1. **Ejecutar todas las celdas hasta la Sección 7** para cargar e inicializar el sistema
2. **Usar los controles superiores** para navegar por diferentes celdas y modelos
3. **Revisar las pestañas inferiores** para análisis detallado

### **Funciones Avanzadas para Análisis Ad-Hoc**

```python
# Ejemplo 1: Análisis rápido de una celda específica
analisis_rapido_celda(aeronave_idx=5, parametro="Peso_Vacio")

# Ejemplo 2: Ver todas las celdas disponibles
celdas = listar_celdas_disponibles()

# Ejemplo 3: Comparar predictores para un parámetro
comparar_predictores("Potencia_Motor")

# Ejemplo 4: Crear visualización 3D (para modelos con 2 predictores)
fig_3d = crear_analisis_3d("aeronave_5_parametro_Peso_Vacio", "linear")

# Ejemplo 5: Exportar reporte completo
exportar_reporte_completo("mi_analisis_completo.xlsx")
```

### **Interpretación de Métricas**

- **MAPE (Mean Absolute Percentage Error)**: 
  - < 3%: Excelente
  - 3-5%: Bueno
  - 5-7.5%: Aceptable
  - > 7.5%: Problemático

- **R² (Coeficiente de Determinación)**:
  - > 0.9: Excelente ajuste
  - 0.8-0.9: Buen ajuste
  - 0.6-0.8: Ajuste aceptable
  - < 0.6: Ajuste insuficiente

- **Confianza (Métrica Combinada)**:
  - > 0.8: Alta confianza
  - 0.6-0.8: Confianza media
  - < 0.6: Baja confianza

### **Flujo de Trabajo Recomendado**

1. **Exploración Inicial**: Usar la interfaz para identificar patrones generales
2. **Análisis Detallado**: Usar funciones avanzadas para celdas específicas
3. **Validación**: Revisar residuos y recomendaciones
4. **Documentación**: Exportar reportes para documentación

### **Troubleshooting**

**Problema**: "No hay modelos disponibles"
- **Solución**: Verificar que los datos se cargaron correctamente y ejecutar la imputación

**Problema**: "Error en visualización 3D"
- **Solución**: Asegurarse de que existen modelos con 2 predictores para el tipo seleccionado

**Problema**: "Exportación falla"
- **Solución**: Verificar permisos de escritura y que no hay archivos abiertos con el mismo nombre

In [11]:
# ================================================================================
# SECCIÓN 9: CONFIGURACIÓN FINAL Y TEST DEL SISTEMA
# ================================================================================

# Configurar opciones de Jupyter para mejor visualización
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# CSS personalizado para mejorar la presentación
css_styles = """
<style>
    .widget-label { font-weight: bold; }
    .widget-dropdown { margin: 5px; }
    .output_wrapper { border: 1px solid #ddd; border-radius: 5px; margin: 10px 0; }
    .output_area { padding: 10px; }
</style>
"""

display(HTML(css_styles))

# Test básico del sistema
def test_sistema_completo():
    """Test básico para verificar que el sistema funciona correctamente."""
    print("🧪 EJECUTANDO TEST DEL SISTEMA")
    print("=" * 35)
    
    try:
        # Test 1: Verificar analizador
        if 'analizador' in globals():
            print("✅ Test 1: Analizador disponible")
        else:
            print("❌ Test 1: Analizador NO disponible")
            return False
        
        # Test 2: Verificar datos cargados
        if hasattr(analizador, 'datos_originales') and analizador.datos_originales is not None:
            print("✅ Test 2: Datos cargados correctamente")
        else:
            print("❌ Test 2: Datos NO cargados")
            return False
        
        # Test 3: Verificar modelos entrenados
        if len(analizador.modelos_por_celda) > 0:
            print(f"✅ Test 3: {len(analizador.modelos_por_celda)} celdas con modelos")
        else:
            print("❌ Test 3: NO hay modelos entrenados")
            return False
        
        # Test 4: Verificar funcionalidades
        try:
            celdas = analizador.obtener_celdas_disponibles()
            if celdas:
                primera_celda = celdas[0]
                info = analizador.obtener_info_celda(primera_celda)
                if info:
                    print("✅ Test 4: Funcionalidades básicas operativas")
                else:
                    print("❌ Test 4: Error en funcionalidades básicas")
                    return False
        except Exception as e:
            print(f"❌ Test 4: Error en funcionalidades - {e}")
            return False
        
        # Test 5: Verificar visualizador
        try:
            visualizador = VisualizadorModelos(analizador)
            print("✅ Test 5: Visualizador creado correctamente")
        except Exception as e:
            print(f"❌ Test 5: Error en visualizador - {e}")
            return False
        
        print("\n🎉 TODOS LOS TESTS PASARON EXITOSAMENTE")
        print("✅ Sistema listo para uso")
        return True
        
    except Exception as e:
        print(f"❌ ERROR GENERAL EN TESTS: {e}")
        return False

# Función de información del sistema
def info_sistema():
    """Mostrar información del estado actual del sistema."""
    print("ℹ️  INFORMACIÓN DEL SISTEMA")
    print("=" * 30)
    
    if 'analizador' in globals():
        print(f"📊 Estado del Analizador:")
        print(f"   • Datos cargados: {'✅' if analizador.datos_originales is not None else '❌'}")
        if analizador.datos_originales is not None:
            print(f"   • Filas de datos: {analizador.datos_originales.shape[0]}")
            print(f"   • Columnas: {analizador.datos_originales.shape[1]}")
        
        print(f"   • Celdas analizadas: {len(analizador.modelos_por_celda)}")
        
        if analizador.modelos_por_celda:
            # Estadísticas de modelos
            total_modelos = 0
            celdas_exitosas = 0
            
            for info in analizador.modelos_por_celda.values():
                for datos_tipo in info["modelos_por_tipo"].values():
                    total_modelos += len(datos_tipo["todos"])
                if info["mejor_global"]:
                    celdas_exitosas += 1
            
            print(f"   • Total modelos entrenados: {total_modelos}")
            print(f"   • Celdas exitosas: {celdas_exitosas}/{len(analizador.modelos_por_celda)}")
            print(f"   • Tasa de éxito: {celdas_exitosas/len(analizador.modelos_por_celda)*100:.1f}%")
    else:
        print("❌ Analizador no disponible")
    
    print(f"\n🛠️  Funcionalidades disponibles:")
    print(f"   • Interfaz interactiva: {'✅' if 'InterfazInteractiva' in globals() else '❌'}")
    print(f"   • Visualizador: {'✅' if 'VisualizadorModelos' in globals() else '❌'}")
    print(f"   • Funciones avanzadas: ✅")

# Ejecutar test automático si hay datos
if 'analizador' in globals():
    print("🚀 VERIFICACIÓN AUTOMÁTICA DEL SISTEMA:")
    print("=" * 40)
    
    sistema_ok = test_sistema_completo()
    
    if sistema_ok:
        print(f"\n📋 RESUMEN DEL SISTEMA:")
        info_sistema()
        
        print(f"\n🎮 COMANDOS ÚTILES:")
        print(f"   • test_sistema_completo() - Verificar estado")
        print(f"   • info_sistema() - Información detallada")
        print(f"   • listar_celdas_disponibles() - Ver celdas")
        print(f"   • analisis_rapido_celda(idx, param) - Análisis rápido")
        
        print(f"\n🎯 ¡SISTEMA COMPLETAMENTE OPERATIVO!")
        print(f"   Use la interfaz interactiva arriba para comenzar el análisis")
    else:
        print(f"\n❌ Sistema no está completamente operativo")
        print(f"   Revise los errores anteriores y ejecute las celdas faltantes")

else:
    print("ℹ️  Para activar el sistema, ejecute las celdas anteriores en orden")

print("\n" + "="*60)
print("🎉 NOTEBOOK DE ANÁLISIS VISUAL DINÁMICO LISTO")
print("="*60)

🚀 VERIFICACIÓN AUTOMÁTICA DEL SISTEMA:
🧪 EJECUTANDO TEST DEL SISTEMA
✅ Test 1: Analizador disponible
✅ Test 2: Datos cargados correctamente
✅ Test 3: 6 celdas con modelos
✅ Test 4: Funcionalidades básicas operativas
✅ Test 5: Visualizador creado correctamente

🎉 TODOS LOS TESTS PASARON EXITOSAMENTE
✅ Sistema listo para uso

📋 RESUMEN DEL SISTEMA:
ℹ️  INFORMACIÓN DEL SISTEMA
📊 Estado del Analizador:
   • Datos cargados: ✅
   • Filas de datos: 10
   • Columnas: 16
   • Celdas analizadas: 6
   • Total modelos entrenados: 240
   • Celdas exitosas: 6/6
   • Tasa de éxito: 100.0%

🛠️  Funcionalidades disponibles:
   • Interfaz interactiva: ✅
   • Visualizador: ✅
   • Funciones avanzadas: ✅

🎮 COMANDOS ÚTILES:
   • test_sistema_completo() - Verificar estado
   • info_sistema() - Información detallada
   • listar_celdas_disponibles() - Ver celdas
   • analisis_rapido_celda(idx, param) - Análisis rápido

🎯 ¡SISTEMA COMPLETAMENTE OPERATIVO!
   Use la interfaz interactiva arriba para comenzar el 

In [12]:
# ================================================================================
# DEMOSTRACIÓN DE FUNCIONALIDADES
# ================================================================================

print("🎯 DEMOSTRACIÓN DE FUNCIONALIDADES AVANZADAS")
print("=" * 45)

# 1. Listar celdas disponibles
print("\n1️⃣ CELDAS DISPONIBLES:")
celdas = listar_celdas_disponibles()

# 2. Análisis rápido de una celda
if celdas:
    # Tomar la primera celda con modelo válido
    for aeronave, parametro, tiene_modelo in celdas:
        if tiene_modelo:
            print(f"\n2️⃣ ANÁLISIS RÁPIDO:")
            analisis_rapido_celda(aeronave, parametro)
            break

# 3. Comparar predictores para un parámetro específico
print(f"\n3️⃣ ANÁLISIS DE PREDICTORES:")
# Buscar un parámetro que aparezca en múltiples celdas
parametros_unicos = list(set([parametro for _, parametro, _ in celdas]))
if parametros_unicos:
    comparar_predictores(parametros_unicos[0])

print(f"\n🎉 DEMOSTRACIÓN COMPLETADA")
print(f"✅ Todas las funcionalidades están operativas")
print(f"🎮 Ahora puede usar la interfaz interactiva completa")

🎯 DEMOSTRACIÓN DE FUNCIONALIDADES AVANZADAS

1️⃣ CELDAS DISPONIBLES:
📋 CELDAS DISPONIBLES PARA ANÁLISIS:
   ✅ Aeronave 5 → payload
   ✅ Aeronave 6 → payload
   ✅ Aeronave 2 → Potencia HP
   ✅ Aeronave 2 → envergadura
   ✅ Aeronave 2 → Alcance de la aeronave
   ✅ Aeronave 2 → Velocidad a la que se realiza el crucero (KTAS)

2️⃣ ANÁLISIS RÁPIDO:
🔍 ANÁLISIS RÁPIDO: Aeronave 5 → payload
🏆 MEJOR MODELO:
   Tipo: linear-2
   MAPE: 2.12%
   R²: 0.995
   Confianza: 0.401
   Predictores: Potencia HP, envergadura

📊 COMPARACIÓN CON OTROS TIPOS:
   Poly: MAPE=5.05%, R²=0.936, Conf=0.368
   Log: MAPE=5.09%, R²=0.978, Conf=0.358
   Pot: MAPE=3.80%, R²=0.988, Conf=0.369
   Exp: MAPE=8.26%, R²=0.927, Conf=0.329

📈 ESTADÍSTICAS:
   Total modelos entrenados: 58
   Tipos disponibles: 5
   Familia de datos usada: sin filtro

3️⃣ ANÁLISIS DE PREDICTORES:
🔍 ANÁLISIS DE PREDICTORES PARA: payload
📊 RANKING DE PREDICTORES:
   Potencia HP:
     • Usado en: 2 modelos
     • MAPE promedio: 2.12%
     • R² promed

In [13]:
# ================================================================================
# FUNCIÓN DE INTEGRACIÓN CON EL FLUJO PRINCIPAL
# ================================================================================

def inicializar_sistema_analisis_desde_main(df_original, diccionarios_modelos, df_resultado_imputacion=None):
    """
    Función principal para inicializar el sistema de análisis desde main.py
    
    Parámetros:
    -----------
    df_original : pd.DataFrame
        DataFrame original con los datos antes de imputación
    diccionarios_modelos : dict
        Diccionario con todos los modelos entrenados, formato:
        {
            "aeronave_X_parametro_Y": {
                "mejor_modelo": dict,
                "todos_los_modelos": list,
                "df_filtrado": pd.DataFrame,
                "predictores": list,
                "familia_usada": str,
                "filtro_aplicado": str
            }
        }
    df_resultado_imputacion : pd.DataFrame, opcional
        DataFrame con los valores imputados
    
    Retorna:
    --------
    InterfazInteractiva: Interfaz lista para usar
    """
    
    print("🔗 INICIANDO INTEGRACIÓN CON FLUJO PRINCIPAL")
    print("=" * 45)
    
    try:
        # Crear analizador
        analizador_main = AnalizadorModelos()
        
        # Cargar datos desde diccionarios
        datos = analizador_main.cargar_desde_diccionarios(
            df_original, 
            diccionarios_modelos, 
            df_resultado_imputacion
        )
        
        if datos is None:
            raise ValueError("No se pudieron cargar los datos desde diccionarios")
        
        # Crear interfaz
        interfaz_main = InterfazInteractiva(analizador_main)
        interfaz_completa = interfaz_main.crear_interfaz_completa()
        
        if interfaz_completa is None:
            raise ValueError("No se pudo crear la interfaz")
        
        print("✅ INTEGRACIÓN EXITOSA")
        print(f"📊 Modelos disponibles: {len(analizador_main.modelos_por_celda)}")
        print("🎮 Interfaz lista para usar")
        
        # Mostrar la interfaz
        display(interfaz_completa)
        
        return interfaz_main
        
    except Exception as e:
        print(f"❌ Error en integración: {e}")
        return None

# Función auxiliar para usar desde main.py
def ejecutar_analisis_visual(df_original, diccionarios_modelos, df_resultado=None):
    """
    Función simplificada para ejecutar desde main.py
    
    Uso en main.py:
    ---------------
    from ADRpy.analisis.analisis_modelos_imputacion import ejecutar_analisis_visual
    
    # Después de ejecutar la imputación completa
    interfaz = ejecutar_analisis_visual(
        df_original=datos_originales,
        diccionarios_modelos=diccionarios_globales,
        df_resultado=df_imputado
    )
    """
    
    # Hacer disponibles los datos en el entorno global para compatibilidad
    globals()['df_original'] = df_original
    globals()['diccionarios_modelos_globales'] = diccionarios_modelos
    if df_resultado is not None:
        globals()['df_resultado_imputacion'] = df_resultado
    
    # Inicializar sistema
    return inicializar_sistema_analisis_desde_main(df_original, diccionarios_modelos, df_resultado)

print("✅ Funciones de integración con main.py disponibles:")
print("   • inicializar_sistema_analisis_desde_main(df, diccionarios, df_resultado)")
print("   • ejecutar_analisis_visual(df, diccionarios, df_resultado)")
print("\n💡 Para usar desde main.py:")
print("   from ADRpy.analisis.analisis_modelos_imputacion import ejecutar_analisis_visual")
print("   interfaz = ejecutar_analisis_visual(df_original, diccionarios_modelos)")

✅ Funciones de integración con main.py disponibles:
   • inicializar_sistema_analisis_desde_main(df, diccionarios, df_resultado)
   • ejecutar_analisis_visual(df, diccionarios, df_resultado)

💡 Para usar desde main.py:
   from ADRpy.analisis.analisis_modelos_imputacion import ejecutar_analisis_visual
   interfaz = ejecutar_analisis_visual(df_original, diccionarios_modelos)


In [17]:
# ========================================================================
# VERIFICACIÓN COMPLETA DEL SISTEMA DE INTEGRACIÓN
# ========================================================================

def verificar_sistema_integracion():
    """
    Verifica que todos los componentes del sistema de integración estén listos.
    """
    
    print("🔍 VERIFICACIÓN DEL SISTEMA DE INTEGRACIÓN")
    print("=" * 50)
    
    errores = []
    advertencias = []
    
    # 1. Verificar módulos necesarios
    print("\n1️⃣ Verificando módulos...")
    
    try:
        import os
        modulos_esperados = [
            "Modulos/imputacion_correlacion_con_diccionarios.py",
            "Modulos/imputation_loop_con_diccionarios.py", 
            "Modulos/integracion_analisis_visual.py"
        ]
        
        for modulo in modulos_esperados:
            if os.path.exists(modulo):
                print(f"  ✅ {modulo}")
            else:
                errores.append(f"Módulo faltante: {modulo}")
                print(f"  ❌ {modulo}")
                
    except Exception as e:
        errores.append(f"Error verificando módulos: {e}")
    
    # 2. Verificar main.py
    print("\n2️⃣ Verificando main.py...")
    
    try:
        with open("main.py", "r", encoding="utf-8") as f:
            contenido_main = f.read()
        
        # Verificar si tiene las modificaciones necesarias
        modificaciones_esperadas = [
            "df_original_para_analisis",
            "bucle_imputacion_similitud_correlacion_con_diccionarios",
            "diccionarios_modelos_globales",
            "generar_diccionarios=True"
        ]
        
        modificaciones_encontradas = 0
        for mod in modificaciones_esperadas:
            if mod in contenido_main:
                print(f"  ✅ {mod}")
                modificaciones_encontradas += 1
            else:
                advertencias.append(f"Modificación faltante en main.py: {mod}")
                print(f"  ❌ {mod}")
        
        if modificaciones_encontradas == 0:
            errores.append("main.py NO ha sido modificado para integración")
        elif modificaciones_encontradas < len(modificaciones_esperadas):
            advertencias.append("main.py tiene modificaciones parciales")
        else:
            print("  ✅ main.py completamente modificado")
            
    except Exception as e:
        errores.append(f"Error verificando main.py: {e}")
    
    # 3. Verificar variables globales desde main.py
    print("\n3️⃣ Verificando variables globales...")
    
    variables_esperadas = [
        "df_original_main",
        "diccionarios_modelos_main", 
        "df_resultado_main",
        "detalles_excel_main"
    ]
    
    variables_encontradas = 0
    for var in variables_esperadas:
        if var in globals():
            tipo_var = type(globals()[var]).__name__
            if hasattr(globals()[var], '__len__'):
                tamaño = len(globals()[var])
                print(f"  ✅ {var}: {tipo_var} (tamaño: {tamaño})")
            else:
                print(f"  ✅ {var}: {tipo_var}")
            variables_encontradas += 1
        else:
            advertencias.append(f"Variable global faltante: {var}")
            print(f"  ❌ {var}")
    
    if variables_encontradas == 0:
        advertencias.append("No hay variables de main.py - ejecutar main.py primero")
    
    # 4. Verificar función de carga
    print("\n4️⃣ Verificando función de carga...")
    
    try:
        if hasattr(analizador, 'cargar_desde_bucle_imputacion'):
            print("  ✅ cargar_desde_bucle_imputacion existe")
            
            if variables_encontradas >= 3:  # Al menos df_original, diccionarios y detalles
                print("  🧪 Probando carga desde variables globales...")
                
                try:
                    resultado = analizador.cargar_desde_bucle_imputacion(
                        globals().get('df_original_main'),
                        globals().get('detalles_excel_main', []),
                        globals().get('diccionarios_modelos_main', {})
                    )
                    
                    if resultado is not None:
                        print("  ✅ Carga desde variables globales exitosa")
                    else:
                        advertencias.append("Carga desde variables globales falló")
                        
                except Exception as e:
                    advertencias.append(f"Error en carga desde variables globales: {e}")
            else:
                advertencias.append("Insuficientes variables globales para probar carga")
        else:
            errores.append("Función cargar_desde_bucle_imputacion no existe")
            
    except Exception as e:
        errores.append(f"Error verificando función de carga: {e}")
    
    # 5. Verificar interfaz
    print("\n5️⃣ Verificando interfaz...")
    
    try:
        if 'interfaz' in globals() and interfaz is not None:
            print("  ✅ Interfaz existe")
        else:
            advertencias.append("Interfaz no creada")
            
        if 'InterfazInteractiva' in globals():
            print("  ✅ Clase InterfazInteractiva disponible")
        else:
            errores.append("Clase InterfazInteractiva no disponible")
            
    except Exception as e:
        errores.append(f"Error verificando interfaz: {e}")
    
    # RESUMEN
    print("\n" + "=" * 50)
    print("📊 RESUMEN DE VERIFICACIÓN")
    print("=" * 50)
    
    if not errores and not advertencias:
        print("✅ SISTEMA COMPLETAMENTE LISTO")
        print("🎮 Todo configurado para análisis visual integrado")
        return True
    else:
        if errores:
            print(f"❌ ERRORES CRÍTICOS ({len(errores)}):")
            for error in errores:
                print(f"   • {error}")
        
        if advertencias:
            print(f"⚠️ ADVERTENCIAS ({len(advertencias)}):")
            for advertencia in advertencias:
                print(f"   • {advertencia}")
        
        print("\n💡 SIGUIENTE PASO:")
        if "main.py NO ha sido modificado" in str(errores):
            print("   1. Modificar main.py siguiendo INSTRUCCIONES_INTEGRACION_MAIN.py")
            print("   2. Ejecutar main.py para generar diccionarios")
            print("   3. Volver a ejecutar esta verificación")
        elif not errores:
            print("   Solo hay advertencias - el sistema puede funcionar")
        
        return False

# Ejecutar verificación
verificar_sistema_integracion()

🔍 VERIFICACIÓN DEL SISTEMA DE INTEGRACIÓN

1️⃣ Verificando módulos...
  ✅ Modulos/imputacion_correlacion_con_diccionarios.py
  ✅ Modulos/imputation_loop_con_diccionarios.py
  ✅ Modulos/integracion_analisis_visual.py

2️⃣ Verificando main.py...
  ❌ df_original_para_analisis
  ❌ bucle_imputacion_similitud_correlacion_con_diccionarios
  ❌ diccionarios_modelos_globales
  ❌ generar_diccionarios=True

3️⃣ Verificando variables globales...
  ❌ df_original_main
  ❌ diccionarios_modelos_main
  ❌ df_resultado_main
  ❌ detalles_excel_main

4️⃣ Verificando función de carga...

5️⃣ Verificando interfaz...
  ✅ Interfaz existe
  ✅ Clase InterfazInteractiva disponible

📊 RESUMEN DE VERIFICACIÓN
❌ ERRORES CRÍTICOS (2):
   • main.py NO ha sido modificado para integración
   • Función cargar_desde_bucle_imputacion no existe
⚠️ ADVERTENCIAS (9):
   • Modificación faltante en main.py: df_original_para_analisis
   • Modificación faltante en main.py: bucle_imputacion_similitud_correlacion_con_diccionarios
  

False

In [18]:
# ========================================================================
# VERIFICACIÓN COMPACTA - PROBLEMAS PRINCIPALES
# ========================================================================

print("🔍 VERIFICACIÓN RÁPIDA DEL ESTADO ACTUAL")
print("=" * 45)

# 1. ¿Existen los módulos?
import os
modulos = [
    "Modulos/imputacion_correlacion_con_diccionarios.py",
    "Modulos/imputation_loop_con_diccionarios.py"
]

print("1️⃣ Módulos creados:")
for mod in modulos:
    existe = "✅" if os.path.exists(mod) else "❌"
    print(f"   {existe} {mod}")

# 2. ¿Está main.py modificado?
print("\n2️⃣ Estado de main.py:")
try:
    with open("main.py", "r") as f:
        main_content = f.read()
    
    if "diccionarios_modelos_globales" in main_content:
        print("   ✅ main.py YA está modificado")
    else:
        print("   ❌ main.py NO está modificado")
        print("   💡 Necesita modificación según INSTRUCCIONES_INTEGRACION_MAIN.py")
except:
    print("   ❌ Error leyendo main.py")

# 3. ¿Hay variables del flujo principal?
print("\n3️⃣ Variables desde main.py:")
vars_main = ["df_original_main", "diccionarios_modelos_main", "df_resultado_main"]
vars_encontradas = 0

for var in vars_main:
    if var in globals():
        print(f"   ✅ {var}")
        vars_encontradas += 1
    else:
        print(f"   ❌ {var}")

if vars_encontradas == 0:
    print("   💡 Ejecutar main.py modificado primero")

# 4. ¿Funciona la carga?
print("\n4️⃣ Función de carga:")
try:
    if hasattr(analizador, 'cargar_desde_bucle_imputacion'):
        print("   ✅ Función existe")
        
        if vars_encontradas > 0:
            print("   🧪 Probando carga...")
            resultado = analizador.cargar_desde_bucle_imputacion(
                globals().get('df_original_main'),
                globals().get('detalles_excel_main', []),
                globals().get('diccionarios_modelos_main', {})
            )
            if resultado is not None:
                print("   ✅ Carga funciona")
            else:
                print("   ❌ Carga falló")
        else:
            print("   ⚠️ Sin variables para probar")
    else:
        print("   ❌ Función no existe")
except Exception as e:
    print(f"   ❌ Error: {e}")

# DIAGNÓSTICO
print("\n" + "=" * 45)
print("🎯 DIAGNÓSTICO:")

if not os.path.exists("Modulos/imputation_loop_con_diccionarios.py"):
    print("❌ PROBLEMA: Módulos no creados")
elif "diccionarios_modelos_globales" not in main_content:
    print("❌ PROBLEMA: main.py no modificado")
elif vars_encontradas == 0:
    print("❌ PROBLEMA: main.py no ejecutado o sin diccionarios")
else:
    print("✅ SISTEMA LISTO para análisis visual")

print("\n💡 SIGUIENTE PASO RECOMENDADO:")
if not os.path.exists("Modulos/imputation_loop_con_diccionarios.py"):
    print("   Los módulos ya fueron creados ✅")
elif "diccionarios_modelos_globales" not in main_content:
    print("   1. Modificar main.py con las instrucciones")
    print("   2. Ejecutar main.py modificado")
    print("   3. Volver al notebook")
elif vars_encontradas == 0:
    print("   1. Ejecutar main.py (ya modificado)")
    print("   2. Volver al notebook")
else:
    print("   1. Crear interfaz con: interfaz = InterfazInteractiva(analizador)")
    print("   2. Mostrar con: interfaz.mostrar_interfaz_completa()")

🔍 VERIFICACIÓN RÁPIDA DEL ESTADO ACTUAL
1️⃣ Módulos creados:
   ✅ Modulos/imputacion_correlacion_con_diccionarios.py
   ✅ Modulos/imputation_loop_con_diccionarios.py

2️⃣ Estado de main.py:
   ❌ main.py NO está modificado
   💡 Necesita modificación según INSTRUCCIONES_INTEGRACION_MAIN.py

3️⃣ Variables desde main.py:
   ❌ df_original_main
   ❌ diccionarios_modelos_main
   ❌ df_resultado_main
   💡 Ejecutar main.py modificado primero

4️⃣ Función de carga:
   ❌ Función no existe

🎯 DIAGNÓSTICO:
❌ PROBLEMA: main.py no modificado

💡 SIGUIENTE PASO RECOMENDADO:
   1. Modificar main.py con las instrucciones
   2. Ejecutar main.py modificado
   3. Volver al notebook
